In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import precision_score, recall_score
from livelossplot import PlotLosses
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import os
import tempfile
import ray
from ray import tune, air
from ray.tune.schedulers import ASHAScheduler
from functools import partial
from ray.train import Checkpoint
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device is {device}!")

local_dir = os.path.abspath("./ray_tune")

Device is cuda!


In [2]:
df = pd.read_csv('./preprocessed.csv', low_memory=False) 
df

,dns_qtype,dns_rcode,dns_query,dst_ip_bytes,src_pkts,label,type,conn_state-OTH,conn_state-REJ,conn_state-RSTO,...,service-ssl,dns_AA--1,dns_AA-F,dns_AA-T,dns_RA--1,dns_RA-F,dns_RA-T,dns_RD--1,dns_RD-F,dns_RD-T
0,0,0,2,3786,6,0,normal,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,12,0,4765,172,1,0,normal,0,0,0,...,0,0,0,1,0,0,1,0,1,0
2,12,3,6112,800,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0
3,43,0,5259,525,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0
4,43,0,2790,408,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46559,1,0,12284,158,1,0,normal,0,0,0,...,0,0,0,1,0,1,0,0,1,0
46560,0,0,2,8462,10,1,ddos,0,0,0,...,1,1,0,0,1,0,0,1,0,0
46561,12,0,3631,375,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0
46562,28,0,9338,712,1,0,normal,0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [3]:
X = df.drop(['label','type'], axis=1)
X.head()

,dns_qtype,dns_rcode,dns_query,dst_ip_bytes,src_pkts,conn_state-OTH,conn_state-REJ,conn_state-RSTO,conn_state-RSTOS0,conn_state-RSTR,...,service-ssl,dns_AA--1,dns_AA-F,dns_AA-T,dns_RA--1,dns_RA-F,dns_RA-T,dns_RD--1,dns_RD-F,dns_RD-T
0,0,0,2,3786,6,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,12,0,4765,172,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
2,12,3,6112,800,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
3,43,0,5259,525,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
4,43,0,2790,408,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [4]:
y = df.filter(items=['label'])
y.head()

,label
0,0
1,0
2,0
3,0
4,0


In [5]:
min_max_scaler = MinMaxScaler()
X =  pd.DataFrame(min_max_scaler.fit_transform(X))
X.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0.000000,0.0,0.000141,0.000044,0.000024,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.047059,0.0,0.336797,0.000002,0.000004,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.047059,0.6,0.432005,0.000009,0.000004,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.168627,0.0,0.371713,0.000006,0.000004,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.168627,0.0,0.197201,0.000005,0.000004,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [8]:
x = torch.Tensor(X.values) 
gt = torch.Tensor(y.values)

In [7]:
gt.size()

torch.Size([46564, 1])

In [9]:
x.size()

torch.Size([46564, 43])

In [10]:
#torch.manual_seed(42)

# Create a TensorDataset from your data and labels
dataset = TensorDataset(x, gt)

# Calculate the number of samples for training and testing datasets
num_samples = len(dataset)
train_size = int(num_samples * 0.7)
test_size = num_samples - train_size

def load_data():
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    return train_dataset, test_dataset

In [10]:
class MLP(nn.Module):
    def __init__(self, l1=100, l2=128, l3=256):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(43, l1)
        self.b1 = nn.BatchNorm1d(l1)
        self.l2 = nn.Linear(l1, l2)
        self.b2 = nn.BatchNorm1d(l2)
        self.l3 = nn.Linear(l2, l3)
        self.b3 = nn.BatchNorm1d(l3)
        self.last = nn.Linear(l3, 1)

    def forward(self, x):
        x = torch.relu(self.b1(self.l1(x)))
        x = torch.relu(self.b2(self.l2(x)))
        x = torch.relu(self.b3(self.l3(x)))
        return torch.sigmoid(self.last(x))

In [10]:
def train_mlp(config):
    net = MLP(config["l1"], config["l2"], config["l3"]).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])

    if "restore" in config:
        checkpoint_path = config["restore"]
        checkpoint = torch.load(checkpoint_path)
        net.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    trainset, _ = load_data()

    test_abs = int(len(trainset) * 0.7)
    train_subset, val_subset = torch.utils.data.random_split(
        trainset, [test_abs, len(trainset) - test_abs])
    
    # Create DataLoaders
    trainloader = DataLoader(train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=0)
    valloader = DataLoader(val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=0)

    for epoch in range(100):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            epoch_steps += 1

        val_loss = 0.0
        val_steps = 0
        total_correct = 0
        total_samples = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                preds = outputs.round()
                total_correct += preds.eq(labels).sum().item()
                total_samples += labels.size(0)
                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        # Calculate overall validation accuracy
        accuracy = (total_correct / total_samples) * 100
                
        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            checkpoint_path = os.path.join(temp_checkpoint_dir, "checkpoint.pt")
            torch.save({
                "model_state_dict": net.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
            }, checkpoint_path)

            # Use ray.train.report to report metrics and checkpoint
            ray.train.report({
                "loss": val_loss,
                "accuracy": accuracy
            }, checkpoint=Checkpoint.from_directory(temp_checkpoint_dir))
    
    print("Finished Training")

In [11]:
def test_best_model(best_result):
    best_trained_model = MLP(best_result.config["l1"], best_result.config["l2"], best_result.config["l3"]).to(device)
    
    print(f'> Number of parameters {len(torch.nn.utils.parameters_to_vector(best_trained_model.parameters()))}')

    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(checkpoint['model_state_dict'])

    _, testset = load_data()
    
    testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = best_trained_model(inputs)
            predicted = outputs.round()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Best trial test set accuracy: {accuracy}%")

In [14]:
def main(num_samples=10, max_num_epochs=100, gpus_per_trial=1):
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(3, 10)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(3, 10)),
        "l3": tune.sample_from(lambda _: 2 ** np.random.randint(3, 10)),
        "lr": tune.loguniform(1e-5, 1e-1),
        "batch_size": tune.choice([64, 128, 256, 512, 1024]),
    }
    scheduler = ASHAScheduler(
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    ray.shutdown()
    ray.init(log_to_driver=False)
    
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_mlp),
            resources={"cpu": 64, "gpu": gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
        run_config=air.RunConfig(
            local_dir=local_dir,
            name="mlp",
        )
    )
    results = tuner.fit()
    
    best_result = results.get_best_result("loss", "min")

    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation loss: {}".format(
        best_result.metrics["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_result.metrics["accuracy"]))

    test_best_model(best_result)
    
main(num_samples=10, max_num_epochs=100, gpus_per_trial=1)

2024-02-22 18:14:22,083	WARNING worker.py:2065 -- Warning: The actor ImplicitFunc is very large (15 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2024-02-22 18:16:33,737	INFO tune.py:1154 -- Total run time: 142.81 seconds (140.98 seconds for the tuning loop).


Best trial config: {'l1': 128, 'l2': 16, 'l3': 64, 'lr': 0.008430868939509882, 'batch_size': 1024}
Best trial final validation loss: 1.1342255622148514
Best trial final validation accuracy: 94.29389508129665
> Number of parameters 9265
Best trial test set accuracy: 89.84967788117395%


In [11]:
y = df.filter(items=['type'])
y['type'].nunique()

10

In [12]:
label_encoder = LabelEncoder()

y_int = label_encoder.fit_transform(y)

/home2/tghv73/myjupyterenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
x = torch.Tensor(X.values) 
gt = torch.Tensor(y_int).long()

In [14]:
#torch.manual_seed(42)

# Create a TensorDataset from your data and labels
dataset = TensorDataset(x, gt)

# Calculate the number of samples for training and testing datasets
num_samples = len(dataset)
train_size = int(num_samples * 0.7)
test_size = num_samples - train_size

In [15]:
class MLP(nn.Module):
    def __init__(self, l1=100, l2=128, l3=256):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(43, l1)
        self.b1 = nn.BatchNorm1d(l1)
        self.l2 = nn.Linear(l1, l2)
        self.b2 = nn.BatchNorm1d(l2)
        self.l3 = nn.Linear(l2, l3)
        self.b3 = nn.BatchNorm1d(l3)
        self.last = nn.Linear(l3, 10)

    def forward(self, x):
        x = torch.relu(self.b1(self.l1(x)))
        x = torch.relu(self.b2(self.l2(x)))
        x = torch.relu(self.b3(self.l3(x)))
        return torch.softmax(self.last(x), dim=1)

In [16]:
def train_mlp(config):
    net = MLP(config["l1"], config["l2"], config["l3"]).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])

    if "restore" in config:
        checkpoint_path = config["restore"]
        checkpoint = torch.load(checkpoint_path)
        net.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    trainset, _ = load_data()

    test_abs = int(len(trainset) * 0.7)
    train_subset, val_subset = torch.utils.data.random_split(
        trainset, [test_abs, len(trainset) - test_abs])
    
    # Create DataLoaders
    trainloader = DataLoader(train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=0)
    valloader = DataLoader(val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=0)

    for epoch in range(100):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            epoch_steps += 1

        val_loss = 0.0
        val_steps = 0
        total_correct = 0
        total_samples = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                _, argmax = torch.max(outputs, dim=1)
                total_correct += argmax.eq(labels).sum().item()
                total_samples += labels.size(0)
                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        # Calculate overall validation accuracy
        accuracy = (total_correct / total_samples) * 100
                
        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            checkpoint_path = os.path.join(temp_checkpoint_dir, "checkpoint.pt")
            torch.save({
                "model_state_dict": net.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
            }, checkpoint_path)

            # Use ray.train.report to report metrics and checkpoint
            ray.train.report({
                "loss": val_loss,
                "accuracy": accuracy
            }, checkpoint=Checkpoint.from_directory(temp_checkpoint_dir))
    
    print("Finished Training")

In [17]:
def test_best_model(best_result):
    best_trained_model = MLP(best_result.config["l1"], best_result.config["l2"], best_result.config["l3"]).to(device)
    
    print(f'> Number of parameters {len(torch.nn.utils.parameters_to_vector(best_trained_model.parameters()))}')

    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(checkpoint['model_state_dict'])

    _, testset = load_data()
    
    testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = best_trained_model(inputs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Best trial test set accuracy: {accuracy}%")

In [19]:
def main(num_samples=10, max_num_epochs=100, gpus_per_trial=1):
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(3, 10)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(3, 10)),
        "l3": tune.sample_from(lambda _: 2 ** np.random.randint(3, 10)),
        "lr": tune.loguniform(1e-5, 1e-1),
        "batch_size": tune.choice([64, 128, 256, 512, 1024]),
    }
    scheduler = ASHAScheduler(
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    ray.shutdown()
    ray.init(log_to_driver=False)
    
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_mlp),
            resources={"cpu": 64, "gpu": gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
        run_config=air.RunConfig(
            local_dir=local_dir,
            name="mlp",
        )
    )
    results = tuner.fit()
    
    best_result = results.get_best_result("loss", "min")

    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation loss: {}".format(
        best_result.metrics["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_result.metrics["accuracy"]))

    test_best_model(best_result)
    
main(num_samples=20, max_num_epochs=100, gpus_per_trial=1)

2024-03-04 19:53:52,747	WARNING worker.py:2065 -- Warning: The actor ImplicitFunc is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2024-03-04 19:57:50,042	INFO tune.py:1154 -- Total run time: 241.13 seconds (240.81 seconds for the tuning loop).


Best trial config: {'l1': 16, 'l2': 64, 'l3': 128, 'lr': 0.0005517260681532147, 'batch_size': 1024}
Best trial final validation loss: 15.352375507354736
Best trial final validation accuracy: 92.53502403108702
> Number of parameters 11818
Best trial test set accuracy: 89.8067287043665%
